In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def data(n): 
    X1 = np.random.multivariate_normal([4, 3], 5*np.eye(2), n)
    X2 = np.random.multivariate_normal([-2, -1], 5*np.eye(2), n)
    X = np.concatenate((X1, X2))
    Y = np.array([0]*n + [1]*n)
    p = np.random.permutation(2*n)
    return X[p].T, Y[p].T